<a href="https://colab.research.google.com/github/WilliamAshbee/3d-synth-data/blob/main/3d2dprepwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

In [62]:
a = np.array([i for i in range(16)])

In [63]:
a = a.reshape(-1,1,1).repeat(16,axis=1).repeat(16,axis=2)

In [64]:
x = a.copy()
y = a.transpose(1,0,2)
z = a.transpose(2,1,0)

In [65]:
x2d = np.zeros((64,64))
y2d = np.zeros_like(x2d)
z2d = np.zeros_like(x2d)


In [55]:
# for i in range(16):
#   for j in range(16):
#     x2d[i*4:(i+1)*4,j*4:(j+1)*4] = i
#     y2d[i*4:(i+1)*4,j*4:(j+1)*4] = j
#     z2d[i*4:(i+1)*4,j*4:(j+1)*4] = np.array([i for i in range(16)]).reshape(4,4)



In [66]:
x3dind = []
y3dind = []
z3dind = []

x2dind = []
y2dind = []

for i in range(16):
  for j in range(16):
    for k in range(16):
      x3dind.append(i)
      y3dind.append(j)
      z3dind.append(k)
      
      x2dind.append(i*4+k%4)
      y2dind.append(j*4+k//4)





In [56]:
x2d[8:12,8:12]

array([[2., 2., 2., 2.],
       [2., 2., 2., 2.],
       [2., 2., 2., 2.],
       [2., 2., 2., 2.]])

In [74]:
x2d[x2dind,y2dind] = x[x3dind,y3dind,z3dind]
y2d[x2dind,y2dind] = y[x3dind,y3dind,z3dind]
z2d[x2dind,y2dind] = z[x3dind,y3dind,z3dind]


In [76]:
for i in range(16):
  for j in range(16):
    print('i,j,val',i,j,x2d[4*i:4*(i+1),j*4:(j+1)*4])
    
for i in range(16):
  for j in range(16):
    print('i,j,val',i,j,y2d[4*i:4*(i+1),j*4:(j+1)*4])

for i in range(16):
  for j in range(16):
    print('i,j,val',i,j,z2d[4*i:4*(i+1),j*4:(j+1)*4])


i,j,val 0 0 [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
i,j,val 0 1 [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
i,j,val 0 2 [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
i,j,val 0 3 [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
i,j,val 0 4 [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
i,j,val 0 5 [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
i,j,val 0 6 [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
i,j,val 0 7 [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
i,j,val 0 8 [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
i,j,val 0 9 [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
i,j,val 0 10 [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
i,j,val 0 11 [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
i,j,val 0 12 [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
i,j,val 0 13 [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 

In [16]:
x2d = x2d.flatten()
y2d = y2d.flatten()
z2d = z2d.flatten()

In [17]:
m2d = np.zeros((64,64))

In [23]:
x2d = x2d.astype(int)
y2d = y2d.astype(int)
z2d = z2d.astype(int)

In [24]:
m2d = a[x2d,y2d,z2d]

In [26]:
m2d.shape

(4096,)

In [31]:
m2d.reshape(64,64)[-4:,-4:]

array([[15, 15, 15, 15],
       [15, 15, 15, 15],
       [15, 15, 15, 15],
       [15, 15, 15, 15]])